In [1]:
import json
import pickle
import os

import pandas as pd
import requests
import numpy as np

import config

data_path = os.path.join("headHunter_data")

text_processing_url = config.text_processing_url

pd.options.display.max_rows = 10
pd.set_option('display.max_columns', None)

In [2]:

with open(os.path.join(data_path, "hh_ids.dat"), 'rb') as inf:
    ids = pickle.load(inf)
    
with open(os.path.join(data_path, "hh_vacancies.dat"), 'rb') as inf:
    vacancies = pickle.load(inf)
    
with open(os.path.join(data_path, "hh_vacancies_ext.dat"), 'rb') as inf:
    vacancies_ext = pickle.load(inf)
    

In [3]:
print(len(vacancies_ext))
print(len(ids))
print(len(set(ids)))

20604
20063
20063


In [ ]:
vac_rows = []
for vac in vacancies:
    try:
        row = {"id": vac["id"], "title": vac["name"],
               "title_normalized": "",
               "title_lemmas": "",
               "title_lemmas_tags": "",
               "title_tokens": "",
               "lang_title": "",
               "requirement_norm": "",
               "requirement_lemmas": "",
               "requirement_lemmas_tags": "",
               "requirement_tokens": "",
               "responsibility_norm": "",
               "responsibility_lemmas": "",
               "responsibility_lemmas_tags": "",
               "responsibility_tokens": "",
               "requirement": vac["snippet"]["requirement"],
               "responsibility": vac["snippet"]["responsibility"],
               "url": vac["url"]}
        vac_rows.append(row) 
    except KeyError:
        print(row)
        None      
vac_df = pd.DataFrame(vac_rows)

print()

vac_rows = []
for i, vac in enumerate(vacancies_ext):
    try:
        row = {"id": vac["id"], "text": vac["description"],
               "text_normalized": "",
               "text_lemmas": "",
               "text_lemmas_tags": "",
               "text_tokens": "",
               "lang_text": "",
               "specializations": [i["name"] for i in vac["specializations"]],
               "profarea_names": [i["profarea_name"] for i in vac["specializations"]]}
        vac_rows.append(row)   
    except KeyError:
        print("Key error, index =", i, "Item =", vacancies_ext[i])
        None
vac_df_ext = pd.DataFrame(vac_rows)

vac_df.drop_duplicates(["id"], inplace=True)
vac_df_ext.drop_duplicates(["id"], inplace=True)
full_df = vac_df_ext.merge(vac_df, left_on='id', right_on='id', how='outer')

print(len(full_df))

### Drop already preprocessed rows

In [7]:
exist_df = pd.read_csv(os.path.join(data_path, "hh_dataset.csv"), sep='\t')
print(exist_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3653 entries, 0 to 3652
Data columns (total 26 columns):
id                            3653 non-null int64
lang_title                    3653 non-null object
requirement                   3536 non-null object
requirement_lemmas            3536 non-null object
requirement_lemmas_tags       3536 non-null object
requirement_norm              3536 non-null object
requirement_tokens            3536 non-null object
responsibility                3472 non-null object
responsibility_lemmas         3472 non-null object
responsibility_lemmas_tags    3472 non-null object
responsibility_norm           3472 non-null object
responsibility_tokens         3472 non-null object
title                         3653 non-null object
title_lemmas                  3335 non-null object
title_lemmas_tags             3335 non-null object
title_normalized              3335 non-null object
title_tokens                  3335 non-null object
url                        

In [8]:
exist_ids = [str(i) for i in exist_df["id"].values]
new_ids = [str(i) for i in full_df["id"].values]
print(len(exist_ids))
print(len(new_ids))

to_drop_ids = list(set(new_ids).intersection(set(exist_ids)))
print(len(to_drop_ids))

to_proc_ids = list(set(new_ids).difference(set(exist_ids)))
print(len(to_proc_ids))

3653
20063
3080
16983


In [9]:
%%time
for i in to_drop_ids:
    full_df = full_df.loc[full_df["id"] != i]  
    #full_df.drop(full_df["id"] == i], inplace=True)
    full_df.reset_index(drop=True, inplace=True)
print(len(full_df))
print(full_df.info())

16983
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16983 entries, 0 to 16982
Data columns (total 26 columns):
id                            16983 non-null object
lang_text                     16981 non-null object
profarea_names                16981 non-null object
specializations               16981 non-null object
text                          16981 non-null object
text_lemmas                   16981 non-null object
text_lemmas_tags              16981 non-null object
text_normalized               16981 non-null object
text_tokens                   16981 non-null object
lang_title                    16983 non-null object
requirement                   16287 non-null object
requirement_lemmas            16983 non-null object
requirement_lemmas_tags       16983 non-null object
requirement_norm              16983 non-null object
requirement_tokens            16983 non-null object
responsibility                15992 non-null object
responsibility_lemmas         16983 non-null object
r

### Normalize text and title using text_preprocessing service

In [10]:
%%time
for index, row in full_df.loc[: 10_000].iterrows():

    requirement = row["requirement"]
    responsibility = row["responsibility"]
    
    fields = ["title", "text"]
    for field in fields:
        text = row[field]
        r = requests.post(text_processing_url + config.STEM_TEXT_PATH,
                          json=text)
        full_df.loc[index, field + "_normalized"] = r.text
        
        r = requests.post(text_processing_url + config.LEMM_TEXT_PATH,
                          json=text)
        full_df.loc[index, field + "_lemmas"] = r.text
        
        r = requests.post(text_processing_url + config.TAG_TEXT_PATH,
                          json=text)
        full_df.loc[index, field + "_lemmas_tags"] = r.text
        
        r = requests.post(text_processing_url + config.TOKEN_TEXT_PATH,
                          json=text)
        full_df.loc[index, field + "_tokens"] = r.text
        
        r = requests.post(text_processing_url + config.DETECT_LANG_PATH ,
                          json=text)
        full_df.loc[index, "lang_" + field] = r.text
                
    fields = ["requirement", "responsibility"]
    for field in fields:
        text = row[field]
        r = requests.post(text_processing_url + config.STEM_TEXT_PATH,
                              json=text)
        full_df.loc[index, field + "_norm"] = r.text
        
        r = requests.post(text_processing_url + config.LEMM_TEXT_PATH,
                          json=text)
        full_df.loc[index, field + "_lemmas"] = r.text
        
        r = requests.post(text_processing_url + config.TAG_TEXT_PATH,
                          json=text)
        full_df.loc[index, field + "_lemmas_tags"] = r.text
        
        r = requests.post(text_processing_url + config.TOKEN_TEXT_PATH,
                          json=text)
        full_df.loc[index, field + "_tokens"] = r.text
    
    if index % 500 == 0:
        print(index)
  

Wall time: 15 s


In [25]:
print(len(full_df))
full_df.tail()

20636


,id,lang_text,lang_title,profarea_names,requirement,requirement_lemmas,requirement_lemmas_tags,requirement_norm,requirement_tokens,responsibility,responsibility_lemmas,responsibility_lemmas_tags,responsibility_norm,responsibility_tokens,specializations,text,text_lemmas,text_lemmas_tags,text_normalized,text_tokens,title,title_lemmas,title_lemmas_tags,title_normalized,title_tokens,url
20631,29103671,russian,russian,"['Управление персоналом, тренинги']",любишь работу рекрутера и готов к новым вызова...,любить работа рекрутер готовый новый вызов зна...,любить_VERB работа_NOUN рекрутер_NOUN готовый_...,люб работ рекрутер гот нов вызов знаеш английс...,любишь работу рекрутера готов новым вызовам зн...,пользоваться сайтами по поиску работы. - имееш...,пользоваться сайт поиск работа иметь хотя конт...,пользоваться_VERB сайт_NOUN поиск_NOUN работа_...,пользова сайт поиск работ имееш хот контакт пе...,пользоваться сайтами поиску работы имеешь хотя...,['Рекрутмент'],<p>Группа рекламных агентств Havas ищет рекрут...,группа рекламный агентство havas искать рекрут...,группа_NOUN рекламный_ADJ агентство_NOUN havas...,групп рекламн агентств havas ищет рекрутераесл...,группа_NOUN рекламный_ADJ агентство_NOUN havas...,Рекрутер,рекрутер,рекрутер_NOUN,рекрутер,рекрутер_NOUN,https://api.hh.ru/vacancies/29103671?host=hh.uz
20632,29103670,russian,russian,"['Бухгалтерия, управленческий учет, финансы пр...","Образование: средне-специальное, высшее. - Уве...",образование высший уверенный владение парсек w...,образование_NOUN высший_ADJ уверенный_ADJ влад...,образован высш уверен владен пк word excel гот...,образование высшее уверенное владение пк word ...,Формирование новых и контроль уже существующих...,формирование новый контроль существовать кальк...,формирование_NOUN новый_ADJ контроль_NOUN суще...,формирован нов контрол существ калькуляцион ка...,формирование новых контроль существующих кальк...,"['Бухгалтер-калькулятор', 'Начальный уровень, ...",<p>Мегамарт - это уральское отделение федераль...,мегамарт этот уральский отделение федеральный ...,мегамарт_NOUN этот_DET уральский_ADJ отделение...,мегамарт эт уральск отделен федеральн торгов с...,мегамарт_NOUN этот_DET уральский_ADJ отделение...,Бухгалтер-калькулятор (улица Луначарского),улица луначарский,улица_NOUN луначарский_ADJ,улиц луначарск,улица_NOUN луначарский_ADJ,https://api.hh.ru/vacancies/29103670?host=hh.uz
20633,27642665,russian,russian,"['Информационные технологии, интернет, телеком']",Знание продукта Microsoft Dynamics CRM (или ин...,знание продукт микрософт dynamics crm иной crm...,знание_NOUN продукт_NOUN микрософт_X dynamics_...,знан продукт microsoft dynamics crm ин crm сис...,знание продукта microsoft dynamics crm иных cr...,Участие в проекте внедрения системы Microsoft ...,участие проект внедрение система микрософт dyn...,участие_NOUN проект_NOUN внедрение_NOUN систем...,участ проект внедрен систем microsoft dynamics...,участие проекте внедрения системы microsoft dy...,['Аналитик'],<p>ПАО «Промсвязьбанк» – один из крупнейших ро...,пао промсвязьбанкать крупный российский банк в...,пао_NOUN промсвязьбанкать_NOUN крупный_ADJ рос...,па промсвязьбанк крупн российск банк входя бан...,пао_NOUN промсвязьбанкать_NOUN крупный_ADJ рос...,Аналитик Dynamics CRM,аналитик dynamics crm,аналитик_NOUN dynamics_X crm_X,аналитик dynamics crm,аналитик_NOUN dynamics_X crm_X,https://api.hh.ru/vacancies/27642665?host=hh.uz
20634,29103668,russian,russian,"['Маркетинг, реклама, PR']",Вам необходимо добиваться поставленного резуль...,необходимый добиваться поставить результат име...,необходимый_ADJ добиваться_VERB поставить_VERB...,необходим добива поставлен результат имет высш...,необходимо добиваться поставленного результата...,Основная обязанность запускать и вести рекламн...,основной обязанность запускать вести рекламный...,основной_ADJ обязанность_NOUN запускать_VERB в...,основн обязан запуска вест рекламн кампан янде...,основная обязанность запускать вести рекламные...,['Интернет-маркетинг'],<p>Ищем 

In [12]:
print(full_df.loc[4, "requirement_lemmas"])
print()
print(full_df.loc[4, "responsibility_lemmas"])
print()
print(full_df.loc[4, "text_tokens"])

высший оконченный образование экономический математический статистика эконометрика либо финансы опыт работа направление инвестиционный анализ крупный компания

систематизация анализ эффективность действовать политика процедура область расчет доход расход дир предприятие разработка предложение изменение

профессиональные обязанности систематизация анализ эффективности действующих политик процедур области расчета доходов расходов дир предприятия разработка предложения изменению указанных политик процедур целью повышения эффективности подготовка презентационных аналитических материалов включая оценку стоимости эффекта внедрения предлагаемых изменений рамках согласования утверждения внедрения предлагаемых изменений обеспечение своевременного качественного внесения изменений внутренние нормативные документы области системы распределения дир разработка новых политик процедур области аллокаций расходов доходов консультирование сотрудников вопросам применения политик процедур анализ рентабельн

In [24]:
print(len(exist_df))
exist_df.head()

3653


,id,lang_title,requirement,requirement_lemmas,requirement_lemmas_tags,requirement_norm,requirement_tokens,responsibility,responsibility_lemmas,responsibility_lemmas_tags,responsibility_norm,responsibility_tokens,title,title_lemmas,title_lemmas_tags,title_normalized,title_tokens,url,lang_text,profarea_names,specializations,text,text_lemmas,text_lemmas_tags,text_normalized,text_tokens
0,29103667,russian,Высшее (желательно техническое) образование. П...,высший желательный технический образование lot...,высший_ADJ желательный_ADJ технический_ADJ обр...,высш желательн техническ образован lotus хорош...,высшее желательно техническое образование lotu...,Ежедневное планирование работы завода (молочно...,ежедневный планирование работа завод молочный ...,ежедневный_ADJ планирование_NOUN работа_NOUN з...,ежедневн планирован работ завод молочн произво...,ежедневное планирование работы завода молочное...,Специаист по планированию производства,специаидать планирование производство,специаидать_NOUN планирование_NOUN производств...,специаист планирован производств,специаидать_NOUN планирование_NOUN производств...,https://api.hh.ru/vacancies/29103667?host=hh.ua,russian,"['Производство', 'Закупки']","['Пищевая промышленность', 'Управление закупка...",<p><strong>Международная компания – крупнейший...,международный компания крупнейший производител...,международный_ADJ компания_NOUN крупнейший_ADJ...,международн компан крупн производител свеж мол...,международный_ADJ компания_NOUN крупнейший_ADJ...
1,29103666,russian,Умение работать в коллективе. Образование не н...,умение работать коллектив образование ниже тех...,умение_NOUN работать_VERB коллектив_NOUN образ...,умен работа коллектив образован ниж техническ ...,умение работать коллективе образование ниже те...,Техническая подготовка выполнения сварочных ра...,технический подготовка выполнение сварочный ра...,технический_ADJ подготовка_NOUN выполнение_NOU...,техническ подготовк выполнен сварочн работ сос...,техническая подготовка выполнения сварочных ра...,Главный сварщик,главный сварщик,главный_ADJ сварщик_NOUN,главн сварщик,главный_ADJ сварщик_NOUN,https://api.hh.ru/vacancies/29103666?host=hh.ua,russian,['Рабочий персонал'],['Сварщик'],<strong>Обязанности:</strong> <ul> <li><em>Тех...,обязанность технический подготовка выполнение ...,обязанность_NOUN технический_ADJ подготовка_NO...,обязан техническ подготовк выполнен сварочн ра...,обязанность_NOUN технический_ADJ подготовка_NO...
2,29103665,russian,At least a bachelor degree in Computer Science...,least bachelor degree computer science master ...,least bachelor degree computer science master ...,least bachelor degre comput scienc master degr...,least bachelor degree computer science master ...,Contributes to product and performance improve...,contributes product performance improvement op...,contributes product performance improvement op...,contribut product perform improv optim exist s...,contributes product performance improvements o...,Java Developer,java developer,java_X developer_X,jav developer,java_X developer_X,https://api.hh.ru/vacancies/29103665?host=hh.ua,english,"['Информационные технологии, интернет, телеком']","['Программирование, Разработка']",<p>Role:<br />Contributes to product and perfo...,role contributes product performance improveme...,role contributes product performance improveme...,role contribut product perform improv optim ex...,role contributes product performance improveme...
3,29103664,russian,Профильное высшее образование (Художественное/...,профильный высший образование креативность нес...,профильный_ADJ высший_ADJ образование_NOUN кре...,профильн высш образован креативн нестандартн м...,профильное высшее образование креативность нес...,Дизайн и верстка еженедельного акционного букл...,дизайн верстка еженедельный акциоть буклет раз...,дизайн_NOUN верстка_NOUN еженедельный_ADJ акци...,дизайн верстк еженедельн акцион буклет разрабо...,дизайн верстка еженедельного акционного буклет...,Дизайнер,дизайнер,дизайнер_NOUN

In [14]:
full_df = full_df.append(exist_df, ignore_index=True, sort=True)


In [16]:
full_df.drop_duplicates(["id"], inplace=True)
print("Final size of dataset =", len(full_df))

Final size of dataset = 20636


In [17]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20636 entries, 0 to 20635
Data columns (total 26 columns):
id                            20636 non-null object
lang_text                     20634 non-null object
lang_title                    20636 non-null object
profarea_names                20634 non-null object
requirement                   19823 non-null object
requirement_lemmas            20519 non-null object
requirement_lemmas_tags       20519 non-null object
requirement_norm              20519 non-null object
requirement_tokens            20519 non-null object
responsibility                19464 non-null object
responsibility_lemmas         20455 non-null object
responsibility_lemmas_tags    20455 non-null object
responsibility_norm           20455 non-null object
responsibility_tokens         20455 non-null object
specializations               20634 non-null object
text                          20634 non-null object
text_lemmas                   20634 non-null object
text_le

In [ ]:
full_df.to_csv(os.path.join("hh_dataset.csv"),
               sep='\t', header=True, index=None)

In [ ]:
vacancies[0]

In [ ]:
vacancies_ext[0]